In [12]:
from astropy.cosmology import Planck18
import astropy.units as u
import lal
import lalsimulation as lalsim
import numpy as np
import os.path as op
import sys
import pandas as pd
import paths
from tqdm import tqdm, trange
import weighting
import scipy.integrate as sint
import intensity_models
from fisher_snrs import compute_snrs
import mock_injections

### mock injections

In [15]:
SENSITIVITIES = {'aLIGO': lalsim.SimNoisePSDaLIGODesignSensitivityP1200087,
                'aplus': lalsim.SimNoisePSDaLIGOAPlusDesignSensitivityT1800042,
                'CE': lalsim.SimNoisePSDCosmicExplorerP1600143}


In [21]:
pe_samples = pd.read_hdf('../../reproduce/selection_samples.h5', 'samples')
pe_samples.keys()

Index(['m1', 'q', 'z', 'a1', 'a2', 'cos_tilt_1', 'cos_tilt_2', 'pdraw_m1sqz',
       'ndraw', 'dm1sz_dm1ddl'],
      dtype='object')

In [ ]:
config_file = "path/to/your/config.yaml"
outfile = "path/to/your/output.json"

In [13]:
config_file = sys.argv[1]
outfile = sys.argv[2]

population_parameters = dict()

with open(config_file) as param_file:
    for line in param_file:
        (key, val) = line.split('=')
        population_parameters[key.strip()] = val.strip()
        try:
            population_parameters[key.strip()] = float(val.strip())
        except ValueError:
            pass

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [ ]:
snr_threshold = population_parameters.pop('snr_threshold', 0)
ndraw = int(population_parameters.pop('ndraw', 1000000))
sensitivity = population_parameters.pop('sensitivity', 'aLIGO')
detectors = population_parameters.pop('detectors', 'H1,L1').split(',')

custom_cosmo = intensity_models.FlatwCDMCosmology(population_parameters['h'], population_parameters['Om'], population_parameters['w'], population_parameters['zmax'])
population_parameters['cosmo'] = custom_cosmo
print("Using the following custom population_parameters: " + str(population_parameters))

In [ ]:
zpdf = ZPDF(lam=population_parameters["lam"], kappa=population_parameters["kappa"], zp=population_parameters["zp"], zmax = population_parameters.get("zmax", 20), cosmo=population_parameters["cosmo"])
    mpdf = PowerLawPDF(1.8, population_parameters["mbh_min"], 400)

    #rng = np.random.default_rng(333165393797366967556667466879860422123)
    rng = np.random.default_rng()

    #df = pd.DataFrame(columns = ['m1', 'q', 'z', 'iota', 'ra', 'dec', 'psi', 'gmst', 's1x', 's1y', 's1z', 's2x', 's2y', 's2z', 'pdraw_mqz', 'SNR_H1', 'SNR_L1', 'SNR_V1', 'SNR'])
    print("drawing zs and ms")
    z = zpdf.icdf(rng.uniform(low=0, high=1, size=ndraw))
    m = mpdf.icdf(rng.uniform(low=0, high=1, size=ndraw))
    print("drawing mts")
    mtpdf = PowerLawPDF(2, m+population_parameters['mbh_min'], 2 * m)

    mt = mtpdf.icdf(rng.uniform(low=0, high=1, size=ndraw))

    m2 = mt - m
    q = m2/m

    print("calculating pdraws")
    pdraw = mpdf(m)*(mtpdf(mt)*m)*zpdf(z)

    m1d = m * (1 + z)
    iota = np.arccos(rng.uniform(low=-1, high=1, size=ndraw))

    ra = rng.uniform(low=0, high=2*np.pi, size=ndraw)
    dec = np.arcsin(rng.uniform(low=-1, high=1, size=ndraw))

    # 0 < psi < pi, uniformly distributed
    psi = rng.uniform(low=0, high=np.pi, size=ndraw)
    gmst = rng.uniform(low=0, high=2*np.pi, size=ndraw)

    print("assigning spins")

    s1x, s1y, s1z = 0,0,0#rng.normal(loc=0, scale=0.2/np.sqrt(3), size=(3,ndraw))
    s2x, s2y, s2z = 0,0,0#rng.normal(loc=0, scale=0.2/np.sqrt(3), size=(3,ndraw))

    print("calculating dLs")

    dm1sz_dm1ddl = weighting.dm1sz_dm1ddl(z, cosmo=population_parameters['cosmo'])
    dL = population_parameters['cosmo'].dL(z)

    df = pd.DataFrame({
        'm1': m,
        'q': q,
        'z': z,
        'dL': dL,
        'm1d': m1d,
        'iota': iota,
        'ra': ra,
        'dec': dec,
        'psi': psi,
        'gmst': gmst,
        's1x': s1x,
        's1y': s1y,
        's1z': s1z,
        's2x': s2x,
        's2y': s2y,
        's2z': s2z,
        'pdraw_mqz': pdraw,
        'dm1sz_dm1ddl': dm1sz_dm1ddl
    })
    if snr_threshold > 0:
        df['SNR'] = compute_snrs(df, detectors=detectors, sensitivity=sensitivity)
    else:
        df['SNR'] = 10000000
    p_pop_numerator = weighting.pop_wt(np.array(df['m1']), np.array(df['q']), np.array(df['z']), default=False, **population_parameters)

    df['p_pop_weight'] = p_pop_numerator / df['pdraw_mqz']
    df['p_pop_numerator'] = p_pop_numerator

    random_number = rng.uniform(low=0, high=1, size = len(p_pop_numerator))
    sel = random_number < (df['p_pop_weight'] / np.max(df['p_pop_weight']))
    population_samples = df[sel]
    df_det = population_samples[population_samples['SNR'] > snr_threshold]

    print(f"Retained {len(df_det)} samples after rejection sampling and applying snr cut.")

    df_det.to_hdf(outfile, key='true_parameters')

    #nex = np.sum(weighting.default_parameters.R*np.exp(weighting.default_log_dNdmdqdV(df_det['m1'], df_det['q'], df_det['z']))*Planck18.differential_comoving_volume(df_det['z']).to(u.Gpc**3/u.sr).value*4*np.pi/(1+df_det['z'])/df_det['pdraw_mqz'])/len(df)
    #nex = calc_nex(df_det, default_settings = default, **population_parameters)
    #print('Found {:d} injections with SNR > {:d}'.format(np.sum(df['SNR'] > snr_threshold), snr_threshold))
    #print('Predicting {:.0f} detections per year'.format(nex))


### mock observations

In [14]:
from astropy.cosmology import Planck18
import astropy.units as u
import dataclasses
from dataclasses import dataclass
import numpy as np
import os.path as op
import pandas as pd
import paths
import seaborn as sns
from tqdm import tqdm
import mock_observations


In [ ]:
rng = np.random.default_rng(181286134409181405721219170031242732711)

    inj = pd.read_hdf(op.join(paths.data, 'mock_injections.h5'), key='true_parameters')

    inj['SNR_OBS'] = inj['SNR'] + rng.normal(loc=0, scale=np.sqrt(3), size=len(inj))

    inj_det = inj[inj['SNR_OBS'] > detection_threshold].copy()
    inj_det['mc'] = inj_det['m1'] * inj_det['q']**(3/5) / (1 + inj_det['q'])**(1/5)
    inj_det['dl'] = Planck18.luminosity_distance(inj_det['z'].to_numpy()).to(u.Gpc).value
    inj_det['mc_det'] = inj_det['mc'] * (1 + inj_det['z'])

    log_mc_obs = []
    sigma_log_mc = []
    q_obs = []
    sigma_q = []
    log_dl_obs = []
    sigma_log_dl = []
    for i, row in tqdm(inj_det.iterrows()):
        uncert = Uncertainties.from_snr(row['SNR_OBS'])
        
        log_mc_obs.append(np.log(row['mc_det']) + uncert.sigma_log_mc*rng.normal())
        sigma_log_mc.append(uncert.sigma_log_mc)

        q_obs.append(row['q'] + uncert.sigma_q*rng.normal())
        sigma_q.append(uncert.sigma_q)

        log_dl_obs.append(np.log(row['dl']) + uncert.sigma_log_dl*rng.normal())
        sigma_log_dl.append(uncert.sigma_log_dl)

    inj_det['log_mc_obs'] = log_mc_obs
    inj_det['sigma_log_mc'] = sigma_log_mc
    inj_det['q_obs'] = q_obs
    inj_det['sigma_q'] = sigma_q
    inj_det['log_dl_obs'] = log_dl_obs
    inj_det['sigma_log_dl'] = sigma_log_dl

    inj_det.to_hdf(op.join(paths.data, 'mock_observations.h5'), key='observations')